In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
%sql
DROP TABLE IF EXISTS default.students;
DROP TABLE IF EXISTS default.students_update;
DROP TABLE IF EXISTS default.enrollments;
DROP TABLE IF EXISTS default.courses;

# Datei einlesen und als Tabelle abspeichern

In [0]:
%python
dataset_school = "/Volumes/workspace/default/volume"

all_files = dbutils.fs.ls(dataset_school)
json_files = [f for f in all_files if f.name.endswith(".json")]

display(json_files)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/volume/courses.json,courses.json,329,1756365194000
dbfs:/Volumes/workspace/default/volume/enrollments.json,enrollments.json,371,1756365193000
dbfs:/Volumes/workspace/default/volume/nested_json.json,nested_json.json,1186,1756365194000
dbfs:/Volumes/workspace/default/volume/students.json,students.json,322,1756365194000
dbfs:/Volumes/workspace/default/volume/students_update.json,students_update.json,216,1756365193000


In [0]:
# JSON lesen:
students_df = spark.read.json("/Volumes/workspace/default/volume/students.json")
students_df.createOrReplaceTempView("students")

studentsupdate_df = spark.read.json("/Volumes/workspace/default/volume/students_update.json")
studentsupdate_df.createOrReplaceTempView("students_update")


courses_df = spark.read.json("/Volumes/workspace/default/volume/courses.json")
courses_df.createOrReplaceTempView("courses")

enrollments_df = spark.read.json("/Volumes/workspace/default/volume/enrollments.json")
enrollments_df.createOrReplaceTempView("enrollments")

In [0]:
# Students speichern
students_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.students")

# Students update
studentsupdate_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.studentsupdate")


# Courses speichern
courses_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.courses")

# Enrollments speichern
enrollments_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.enrollments")

![Bildschirmfoto 2025-08-09 um 09.30.21.png](./Bildschirmfoto 2025-08-09 um 09.30.21.png "Bildschirmfoto 2025-08-09 um 09.30.21.png")

In [0]:
%sql
-- Daten lesen
SELECT * FROM default.students;

email,gpa,profile,student_id,updated
anna@example.com,3.9,full-time,1,2025-08-01
ben@example.com,3.2,part-time,2,2025-08-03
clara@example.com,3.7,exchange,3,2025-08-06


In [0]:
%sql
-- Daten lesen
SELECT * FROM default.courses;

category,course_id,instructor,price,title
Technology,101,Dr. Smith,850,Data Engineering
AI,102,Dr. Lee,1200,Machine Learning
Philosophy,103,Dr. Kim,600,Ethics in AI


In [0]:
%sql
-- Daten lesen
SELECT * FROM default.enrollments;

courses,enroll_id,quantity,student_id,timestamp,total
List(101),e001,1,1,2025-08-01T09:00:00,850
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800
List(103),e003,1,3,2025-08-03T11:30:00,600


# Join-Operationen in Spark SQL

In [0]:
%sql
-- Array auf einzelne Zeilen aufbrechen und joinen
CREATE OR REPLACE VIEW enrollments_enriched AS
SELECT 
    e.enroll_id,
    e.student_id,
    e.timestamp,
    e.total,
    c.course_id,
    c.title,
    c.instructor
FROM (
    SELECT 
        enroll_id,
        student_id,
        timestamp,
        total,
        explode(courses) AS course_id -- Strucktur aufbrechen mit explode z.B. [102,103]
    FROM default.enrollments
) e
INNER JOIN default.courses c
ON e.course_id = c.course_id;

-- Ergebnis anzeigen
SELECT * FROM enrollments_enriched;

enroll_id,student_id,timestamp,total,course_id,title,instructor
e001,1,2025-08-01T09:00:00,850,101,Data Engineering,Dr. Smith
e002,2,2025-08-02T10:00:00,1800,102,Machine Learning,Dr. Lee
e002,2,2025-08-02T10:00:00,1800,103,Ethics in AI,Dr. Kim
e003,3,2025-08-03T11:30:00,600,103,Ethics in AI,Dr. Kim


Set-Operationen in Spark SQL

In [0]:
%sql
SELECT * FROM default.studentsupdate

email,gpa,profile,student_id,updated
hans@example.com,1.9,full-time,4,2025-11-01
anton@example.com,6.0,part-time,5,2025-08-03


In [0]:
%sql
SELECT * FROM students
UNION
SELECT *FROM studentsupdate

email,gpa,profile,student_id,updated
ben@example.com,3.2,part-time,2,2025-08-03
anna@example.com,3.9,full-time,1,2025-08-01
clara@example.com,3.7,exchange,3,2025-08-06
anton@example.com,6.0,part-time,5,2025-08-03
hans@example.com,1.9,full-time,4,2025-11-01


# Higher-Order Funktionen

In [0]:
%sql
--Ausgangslage
SELECT * from default.enrollments

courses,enroll_id,quantity,student_id,timestamp,total
List(101),e001,1,1,2025-08-01T09:00:00,850
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800
List(103),e003,1,3,2025-08-03T11:30:00,600


In [0]:
%sql
-- Beispiel FILTER: nur bestimmte Kurs-IDs behalten (z. B. 103)
SELECT
  enroll_id,
  courses,
  FILTER(courses,
         c -> c = 103) AS only_103 -- FILTER(array, x -> predicate) wählt Elemente aus einem Array.
FROM enrollments
WHERE size(FILTER(courses, c -> c = 103)) > 0; -- WHERE ist optional um leere Arrays austzuschliessen

enroll_id,courses,only_103
e002,"List(102, 103)",List(103)
e003,List(103),List(103)


In [0]:
%sql
-- Beispiel TRANSFORM: Gesamtpreis pro Enrollment auf Kurse verteilen
SELECT
  enroll_id,
  courses,
  TRANSFORM( -- TRANSFORM(array, x -> expr) bildet jedes Element auf einen neuen Wert/STRUCT ab.
    courses,
    c -> named_struct(
           'course_id', c,
           'allocated_total', ROUND(total / size(courses), 2)
         )
  ) AS per_course_allocation
FROM enrollments;

enroll_id,courses,per_course_allocation
e001,List(101),"List(List(101, 850.0))"
e002,"List(102, 103)","List(List(102, 900.0), List(103, 900.0))"
e003,List(103),"List(List(103, 600.0))"


# SQL user-defined functions (UDFs)


In [0]:
%sql
-- Beispiel 1
-- GPA (0–6.0) → Prozent (0–100)
CREATE OR REPLACE FUNCTION gpa_to_percentage(g DOUBLE)
RETURNS INT
RETURN CAST(ROUND(g * 16.66666667) AS INT); -- Höchster GPA ist 6.0. 1/6 = 16.6666667

-- Nutzung
SELECT student_id, gpa, gpa_to_percentage(gpa) AS pct
FROM default.students
UNION -- wir nehmen auch die neuen Studenten gleich mit dazu
SELECT student_id, gpa, gpa_to_percentage(gpa) AS pct 
FROM default.studentsupdate;

student_id,gpa,pct
3,3.7,62
2,3.2,53
1,3.9,65
5,6.0,100
4,1.9,32


In [0]:
%sql
-- Beispiel 2
CREATE OR REPLACE FUNCTION get_letter_grade(g DOUBLE)
RETURNS STRING
RETURN CASE
         WHEN g >= 5.5  THEN 'A'
         WHEN g >= 3.9 THEN 'B'
         WHEN g >= 3.42 THEN 'C'
         ELSE 'F'
       END;

SELECT student_id, gpa, get_letter_grade(gpa) AS grade
FROM default.students
UNION
SELECT student_id, gpa, get_letter_grade(gpa) AS grade 
FROM default.studentsupdate
;

student_id,gpa,grade
1,3.9,B
2,3.2,F
3,3.7,C
4,1.9,F
5,6.0,A


In [0]:
%sql
DESCRIBE FUNCTION gpa_to_percentage;

function_desc
Function: workspace.default.gpa_to_percentage
Type: SCALAR
Input: g DOUBLE
Returns: INT


In [0]:
%sql
DESCRIBE FUNCTION EXTENDED get_letter_grade;

function_desc
Function: workspace.default.get_letter_grade
Type: SCALAR
Input: g DOUBLE
Returns: STRING
Deterministic: true
Data Access: CONTAINS SQL
Configs: spark.connect.session.connectML.enabled=false
spark.databricks.docingest.client.v2.enabled=true
spark.databricks.sql.expression.aiFunctions.repartition=0
spark.databricks.sql.functions.aiForecast.enabled=false


# Aufräumen

In [0]:
%sql
DROP FUNCTION IF EXISTS gpa_to_percentage;
DROP FUNCTION IF EXISTS get_letter_grade;

In [0]:
%sql
DROP TABLE IF EXISTS default.courses;
DROP TABLE IF EXISTS default.enrollments;
DROP TABLE IF EXISTS default.students;
DROP TABLE IF EXISTS default.studentsupdate;

In [0]:
%sql
DROP VIEW IF EXISTS default.enrollments_enriched;